In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Step 1, 2 데이터 다운로드 및 읽어오기

In [2]:
import glob
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

txt_file_path = './lyricist/data/lyrics/*.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


# Step 3. 데이터 정제

In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [30]:
corpus = []

for sentence in raw_corpus:
    new_sentence = preprocess_sentence(sentence)
    if len(new_sentence) == 0: continue
    if new_sentence[-1] == ":": continue
    
    # if new_sentence.count(" ") <= 13:
    # if len(new_sentence.split()) <= 15: # without corpus having more than 15 token
    #    corpus.append(new_sentence)
    corpus.append(preprocess_sentence(sentence))
    
        
corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

In [75]:
new_corpus = []
for i in range(len(corpus) - 1):
    if len(corpus[i].split()) <= 13: # <Start> & <End>
        new_corpus.append(corpus[i])

In [76]:
len(corpus)

187088

In [77]:
len(new_corpus)

153375

In [78]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(new_corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(new_corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   72  254 ...    0    0    0]
 [   2    4   52 ...    0    0    0]
 [   2    4 1019 ...    0    0    0]
 ...
 [   2  221    1 ...    0    0    0]
 [   2    1    4 ...    0    0    0]
 [   2  683    5 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f41b6b48f90>


In [79]:
tensor

array([[   2,   72,  254, ...,    0,    0,    0],
       [   2,    4,   52, ...,    0,    0,    0],
       [   2,    4, 1019, ...,    0,    0,    0],
       ...,
       [   2,  221,    1, ...,    0,    0,    0],
       [   2,    1,    4, ...,    0,    0,    0],
       [   2,  683,    5, ...,    0,    0,    0]], dtype=int32)

In [80]:
len(tensor[0])

15

In [81]:
tokenizer

In [82]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  72 254   4  52 705   3   0   0   0   0   0   0   0]
[ 72 254   4  52 705   3   0   0   0   0   0   0   0   0]


# Step 4. 평가 데이터셋 분리

In [83]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, shuffle = True)

In [84]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [85]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (122700, 14)
Target Train: (122700, 14)


# Step 5. 인공지능 만들기

In [86]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [87]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.2625084e-04, -6.2304775e-06, -9.4458475e-05, ...,
          2.1256608e-04, -1.9971860e-04,  4.3399596e-05],
        [ 1.8461673e-04, -2.4515168e-06, -3.2792447e-04, ...,
          8.6879474e-05, -4.9097760e-04,  1.9980471e-04],
        [ 3.7683416e-04,  9.1476097e-05, -2.8378403e-04, ...,
          7.0618029e-05, -3.0648970e-04,  8.8869645e-05],
        ...,
        [ 3.7264646e-04, -1.2789902e-04,  1.7314256e-03, ...,
          5.0077244e-04, -5.0454371e-04, -1.6534987e-03],
        [ 5.7094829e-04, -9.6801850e-05,  1.9770933e-03, ...,
          6.9885032e-04, -6.5774797e-04, -1.9571872e-03],
        [ 7.7310740e-04, -7.5450698e-05,  2.1906572e-03, ...,
          8.7160681e-04, -8.1312074e-04, -2.2199615e-03]],

       [[ 1.2625084e-04, -6.2304775e-06, -9.4458475e-05, ...,
          2.1256608e-04, -1.9971860e-04,  4.3399596e-05],
        [ 3.8278333e-04,  3.8775954e-05, -4.2555443e-04, ...,
          1.2292201e-04, 

In [88]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [89]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
479/479 [==============================] - 58s 121ms/step - loss: 3.1143
Epoch 2/30
479/479 [==============================] - 59s 123ms/step - loss: 2.6820
Epoch 3/30
479/479 [==============================] - 60s 125ms/step - loss: 2.5387
Epoch 4/30
479/479 [==============================] - 61s 128ms/step - loss: 2.4258
Epoch 5/30
479/479 [==============================] - 63s 132ms/step - loss: 2.3276
Epoch 6/30
479/479 [==============================] - 64s 133ms/step - loss: 2.2391
Epoch 7/30
479/479 [==============================] - 64s 133ms/step - loss: 2.1579
Epoch 8/30
479/479 [==============================] - 64s 133ms/step - loss: 2.0811
Epoch 9/30
479/479 [==============================] - 63s 132ms/step - loss: 2.0069
Epoch 10/30
479/479 [==============================] - 64s 133ms/step - loss: 1.9357
Epoch 11/30
479/479 [==============================] - 64s 133ms/step - loss: 1.8685
Epoch 12/30
479/479 [==============================] - 63s 133ms/step - lo

In [92]:
# 모델 검증

results = model.evaluate(enc_val, dec_val, verbose = 2)
print(results)

959/959 - 8s - loss: 2.1939
2.193932056427002


In [90]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [91]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len = 20)

'<start> i m not the only one <end> '

# 회고

### - 이번 프로젝트에서 어려웠던 점.
    역시나 자연어처리쪽은 어렵다는 것을 다시 한번 느끼게 되었습니다. (나랑 안맞아..)
    RNN 계열의 모델 원리를 이해하고 구성해보는 과정에서 저번 CNN 때와 마찬가지로 알고리즘 자체에 대한 깊은 이해가 전제된다면 전반적인 processing이 더 잘 이해될 것 같다는 느낌이 들었습니다. 
    
    
### - 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.
    처음에 dataset을 split 하지 않고 학습을 진행해보았는데, 예상했던 대로 dataset의 크기가 클 수록 loss 측면에서 더 나은 학습을 하는 것 같았습니다.
    현재 모델의 loss가 trainset에 대해서는 1.0 초반대가 나오고, testset에 대해서는 2.19 정도가 나오는데, 아마 데이터셋의 크기 자체가 더 커지게 된다면 loss 자체를 더 줄일 수 있을 것으로 생각됩니다.
    
    RNN 알고리즘이 어떻게 동작하는지는 어느정도 이해를 하게 되었습니다. 하지만, 단순히 확률값에 근거해서 다음 단어를 예측하는 방식이 어떻게 GPT-3가 사람이 쓰는 문장처럼 자연스러운 문장을 구사하게 되는 건지는 잘 모르겠습니다. (뭔가 일률적인 문장이 생성되어야 할 거 같은데 또 그렇지 않은거 보면 뭔가 추가적으로 학습이 되는 무언가가 있는거 같기도 하고..?!)
    
    패딩을 하는 과정에서 pre / post 두개의 방식이 있는 것을 이전 노드에서 학습해서 알고있었는데, LSTM, GRU(순차적 진행방식)에서 pre-padding인 경우에 성능이 더 좋다고 배웠습니다. 그치만, 이번 노드에서는 post-paddinig 방식이 적용되었는데, 그 이유가 다음 단어의 예측을 통해 학습을 하는 것이기 때문에 뒷부분에 0으로 padding하는 것이 더 낫다고 생각을 했지만, 다시 생각해보면 LSTM 모델을 적용한건데, pre-padding 방식이 더 나은것이 아닐까 하는 생각을 하게 되었습니다. (pre로도 한번 진행을 해봐야 할 것 같다!)
    
    
    
### - 루브릭 평가 지표를 맞추기 위해 시도한 것들.
    1) 가사 텍스트 생성 모델이 정상적으로 동작하는가? (텍스트 제네레이션 결과가 그럴듯한 문장으로 생성되는가?)
    네. 처음 모델 학습이후 생성한 문장은 '<start> i m the one , yeah , i m the one <end> ' 이었고, 이후 추가적인 학습을 통해 생성한 문장은 '<start> i m not the only one <end> '이었습니다. 두 문장 모두 크게 이상하지 않은 문장이라고 생각됩니다.
    
    
    2) 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? (특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행되었는가?)
    네. 특수문자 제거, 토크나이저 생성, 패딩 처리 모두 수행하였고, 데이터셋을 split해서 trainset으로 모델을 학습(loss : 1.03)시키고, 해당 모델에 대한 검증을 testset으로 확인(loss : 2.19)하였습니다.
    
    
    3) 텍스트 생성모델이 안정적으로 학습되었는가? (텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?)
    네. val loss가 2.2 이하로 낮아진 것을 확인하였습니다. (정상적으로 학습이 진행된 것 같습니다.)
    
    